In [1]:
!python --version

Python 3.10.8


previous config to the test:

In [2]:
#autoreload permite que jupyter vea las modif de los script de forma automática
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from scipy.integrate import odeint

In [4]:
def pend(y, t, b, λ):
    θ, ω = y
    dydt = [ω, -b * ω - λ * np.sin(θ)]

    return dydt

In [5]:
from skreducedmodel.reducedbasis import select_child_node

----

test config and run:

In [6]:
b = 0.2
y0 = [np.pi / 2, 0.0]

train_parameters = np.linspace(1, 5, 101)
test_parameters = np.linspace(1, 5, 100)[1:-1]

times = np.linspace(0, 50, 1001)

training_set = []
for λ in train_parameters:
    sol = odeint(pend, y0, times, (b, λ))
    training_set.append(sol[:, 0])
training_set = np.array(training_set)

test_set = []
for λ in test_parameters:
    sol = odeint(pend, y0, times, (b, λ))
    test_set.append(sol[:, 0])
test_set = np.array(test_set)

In [7]:
from skreducedmodel.reducedbasis import ReducedBasis

nmax = 5

model = ReducedBasis(index_seed_global_rb=0,
                     greedy_tol=1e-16,
                     lmax=1,
                     nmax=nmax,
                     normalize=True
                    )

model.fit(training_set=training_set,
          parameters=train_parameters,
          physical_points=times
         )

In [8]:
from skreducedmodel.reducedbasis import normalize_set, error

# normalizamos el conjunto de ondas de testeo
test_set_normalized = normalize_set(test_set, times)

errors = []
for h, q in zip(test_set_normalized, test_parameters):
    # calculamos la proyección de las ondas con el modelo hp-greedy
    ht = model.transform(h,q)
    # calculamos el error para la proyección
    errors.append(error(h, ht, times))

In [11]:
model.tree.height

1

In [12]:
node = model.tree
node.train_parameters[node.idx_anchor_0],node.idx_anchor_0

(1.0, 0)

In [13]:
node.train_parameters[node.idx_anchor_1],node.idx_anchor_1

(5.0, 100)

In [21]:
test_parameters_ones = np.ones(5)*3
times = np.linspace(0, 50, 1001)

test_set_ones = []
for λ in test_parameters_ones:
    sol = odeint(pend, y0, times, (b, λ))
    test_set_ones.append(sol[:, 0])
test_set_ones = np.array(test_set_ones)

In [39]:
model.transform(test_set_ones, test_parameters_ones)

node name (0, 1)


array([[ 1.55108277,  1.54736984,  1.53628049, ..., -0.00752865,
        -0.00690791, -0.00624157],
       [ 1.55108277,  1.54736984,  1.53628049, ..., -0.00752865,
        -0.00690791, -0.00624157],
       [ 1.55108277,  1.54736984,  1.53628049, ..., -0.00752865,
        -0.00690791, -0.00624157],
       [ 1.55108277,  1.54736984,  1.53628049, ..., -0.00752865,
        -0.00690791, -0.00624157],
       [ 1.55108277,  1.54736984,  1.53628049, ..., -0.00752865,
        -0.00690791, -0.00624157]])

In [42]:
for sample, parameter in zip(test_set_ones, test_parameters_ones):
    print(model.transform(sample, parameter))

node name (0, 0)
[ 1.57079633  1.56705879  1.55589593 ... -0.0099507  -0.00975695
 -0.00949237]
node name (0, 0)
[ 1.57079633  1.56705879  1.55589593 ... -0.0099507  -0.00975695
 -0.00949237]
node name (0, 1)
[ 1.55108277  1.54736984  1.53628049 ... -0.00752865 -0.00690791
 -0.00624157]
node name (0, 0)
[ 1.57079633  1.56705879  1.55589593 ... -0.0099507  -0.00975695
 -0.00949237]
node name (0, 1)
[ 1.55108277  1.54736984  1.53628049 ... -0.00752865 -0.00690791
 -0.00624157]


los dos metodos de arriba dan diferente (evaluan en rb diferentes)

reiniciar las evaluaciones de una forma limpia?

ie: como controlo reiniciar la semilla con la cual elijo para que lado va la onda de test?

diferencias entre las ultimas dos formas

mejor transform que reciba una lista de las ondas a evaluar?